In [6]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)

import asyncio
import multiprocessing as mp
import os
import time
from datetime import datetime
from functools import partial
from typing import Dict, TypeAlias

import httpx
import numpy as np
import pandas as pd
import ujson as json
from concurrent.futures import ProcessPoolExecutor, as_completed

from RL_BondPricer import RL_BondPricer
from QL_BondPricer import QL_BondPricer
from script import FedInvestFetcher

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [7]:
curve_data_fetcher = FedInvestFetcher(use_ust_issue_date=True, no_logs_plz=True)

In [10]:
raw_auctions_df = curve_data_fetcher.get_auctions_df()
raw_auctions_df

,cusip,security_type,auction_date,issue_date,maturity_date,price_per100,allocation_pctage,avg_med_yield,bid_to_cover_ratio,comp_accepted,...,security_term,original_security_term,security_term_week_year,primary_dealer_accepted,primary_dealer_tendered,reopening,total_accepted,total_tendered,treas_retail_accepted,treas_retail_tenders_accepted
218,912797LF2,Bill,2024-09-03,2024-09-05,2024-12-05,null,null,null,null,null,...,13-Week,26-Week,13-Week,null,null,Yes,null,null,null,Yes
217,912797MM6,Bill,2024-09-03,2024-09-05,2025-03-06,null,null,null,null,null,...,26-Week,26-Week,26-Week,null,null,No,null,null,null,Yes
216,912797KU0,CMB,2024-09-03,2024-09-05,2024-10-17,null,null,null,null,null,...,42-Day,26-Week,0-Week,null,null,Yes,null,null,null,No
215,912797MH7,Bill,2024-09-03,2024-09-05,2025-09-04,null,null,null,null,null,...,52-Week,52-Week,52-Week,null,null,No,null,null,null,Yes
214,912797LV7,Bill,2024-08-29,2024-09-03,2024-10-29,99.206667,6.850000,null,2.600000,78396650000,...,8-Week,17-Week,8-Week,35573675000,153810000000,Yes,80241809600,208140459600,1104339700,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,9127935W2,Bill,1980-01-02,1980-01-08,1981-01-02,null,3.000000,null,null,null,...,52-Week,52-Week,52-Week,null,null,No,4007825000,6329685000,null,null
2,9127933Y0,Bill,1979-12-28,1980-01-03,1980-04-03,null,21.000000,null,null,null,...,13-Week,null,13-Week,null,null,Yes,3257440000,5371540000,null,null
3,9127934U7,Bill,1979-12-28,1980-01-03,1980-07-03,null,94.000000,null,null,null,...,26-Week,26-Week,26-Week,null,null,No,3348725000,5954350000,null,null
1,912810CK2,Bond,1979-11-01,1979-11-15,2009-11-15,null,44.000000,10.440000,null,null,...,30-Year,30-Year,30-Year,null,null,No,2315000000,3594000000,null,null
